In [155]:
import pyodbc 
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules



pd.set_option('display.max_colwidth', None)
# Define connection parameters
server = 'localhost'
database = 'CompanyX'

# Establish a connection with Windows Authentication
connection_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

try:
    conn = pyodbc.connect(connection_string)
    print ("Connected Successfully")
except:
     print ("Connection failed")

# Your SQL query
sql_query = 'SELECT * FROM [CompanyX].[dbo].[Sales_Facts]'

# Execute the query and fetch results into a DataFrame
df = pd.read_sql(sql_query, conn)






# Define a custom aggregation function
def custom_agg(series):
    return ', '.join(map(str, series))

# Combine values based on ID
selected_cols_df = df.groupby('SalesOrderID')['ProductID'].agg(custom_agg).reset_index().head(10)
1# Display the result
display(selected_cols_df)



#top 10 selling products


# Step 1: Group by ProductID and sum the OrderQty
product_sales = df.groupby('ProductID')['OrderQty'].sum()

# Step 2: Sort the products by total sales in descending order
sorted_products = product_sales.sort_values(ascending=False)

# Step 3: Take the top 10 most sold products
top_10_products = sorted_products.head(10)

# Convert the result to a DataFrame for presentation
top_10_products_df = pd.DataFrame({'ProductID': top_10_products.index, 'OrderQty': top_10_products.values})

# Display the top 10 products in a table format
display(top_10_products_df)

included_products = ['712', '870', '711', '715', '708', '707','864','873','884','714']
included_products_float = [float(product) for product in included_products]

df1 = pd.get_dummies(df.set_index('SalesOrderID')['ProductID'],dtype=float)
df1 = df1[included_products_float].max(level=0).reset_index()
print (df1)


# Display the intermediate bina



Connected Successfully


,SalesOrderID,ProductID
0,43659,"776, 777, 778, 771, 772, 773, 774, 714, 716, 709, 712, 711"
1,43660,"762, 758"
2,43661,"745, 743, 747, 712, 715, 742, 775, 778, 711, 741, 776, 773, 716, 777, 708"
3,43662,"764, 770, 730, 754, 725, 762, 765, 768, 753, 756, 763, 732, 758, 729, 722, 749, 760, 726, 733, 738, 766, 755"
4,43663,760
5,43664,"772, 775, 714, 716, 777, 771, 773, 778"
6,43665,"711, 773, 707, 715, 777, 712, 775, 778, 709, 776"
7,43666,"764, 753, 732, 756, 768, 766"
8,43667,"710, 773, 778, 775"
9,43668,"756, 753, 760, 765, 715, 730, 707, 711, 754, 712, 729, 755, 761, 770, 726, 764, 766, 725, 716, 768, 732, 758, 762, 738, 714, 722, 708, 733, 763"


,ProductID,OrderQty
0,712,8306.0
1,870,6815.0
2,711,6740.0
3,715,6582.0
4,708,6528.0
5,707,6261.0
6,864,4247.0
7,873,3865.0
8,884,3864.0
9,714,3634.0


       SalesOrderID  712  870  711  715  708  707  864  873  884  714
0             43659  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
1             43660  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2             43661  1.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0
3             43662  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4             43663  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
...             ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
31460         75119  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
31461         75120  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
31462         75121  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
31463         75122  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
31464         75123  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[31465 rows x 11 columns]


In [160]:
# Load necessary libraries
import pyodbc
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules



# Group and pivot the data
basket = (df
          .groupby(['SalesOrderID', 'ProductID'])['OrderQty']
          .sum().unstack().reset_index().fillna(0)
          .set_index('SalesOrderID'))

# Convert quantities to binary values
def hot_encode(x):
    if(x <= 0):
        return 0
    if(x >= 1):
        return 1

basket_encoded = basket.applymap(hot_encode)
basket = basket_encoded

# Run Apriori algorithm
frq_items = apriori(basket, min_support=0.03, use_colnames=True)

# Collect the inferred rules in a dataframe
rules = association_rules(frq_items, metric="lift", min_threshold=1)
rules = rules.sort_values(['confidence', 'lift'], ascending=[False, False])

# Display the rules
print(rules)

   antecedents consequents  antecedent support  consequent support   support  \
12       (872)       (870)            0.054410            0.148991  0.048339   
11       (871)       (870)            0.064357            0.148991  0.053774   
7        (715)       (712)            0.051867            0.107421  0.037184   
15       (930)       (921)            0.044367            0.098363  0.030415   
10       (870)       (871)            0.148991            0.064357  0.053774   
6        (712)       (715)            0.107421            0.051867  0.037184   
3        (708)       (712)            0.095503            0.107421  0.031336   
5        (711)       (712)            0.098141            0.107421  0.032067   
13       (870)       (872)            0.148991            0.054410  0.048339   
1        (707)       (712)            0.097918            0.107421  0.030288   
14       (921)       (930)            0.098363            0.044367  0.030415   
8        (712)       (870)            0.